In [8]:
import numpy as np
from pandas import DataFrame, read_csv
import pandas as pd 
from scipy.sparse import lil_matrix

file = './../dataset/anonymous-msweb.data'

In [9]:
raw_df = pd.read_csv(file,names = ["category", "value",'other-1','other-2'])
raw_df = raw_df[['category','value']]
raw_df = raw_df[(raw_df.category == 'C') | (raw_df.category == 'V')]

users = raw_df[(raw_df.category == 'C')].value.to_numpy()
items = raw_df[(raw_df.category == 'V')].value.to_numpy()
items = list(set(items))
users = list(set(users))
items.sort()
users.sort()

raw_df['cumsum'] = 0

raw_df.loc[raw_df['category'] == 'C', 'cumsum'] = [1]

raw_df['cumsum'] = raw_df['cumsum'].cumsum()

values = raw_df[(raw_df.category == 'V')]

values['user'] = values['cumsum'].apply(lambda x: users[x-1])
values['item'] = values['value']
values = values[['user', 'item']]

n = len(users)
m = len(items)

print(values)
sparse_mat =  np.zeros((n,m))

for index, row in values.iterrows():
    sparse_mat[users.index(row['user']), items.index(row['item'])] = 1

items = list(map(lambda x: "X"+str(x), items))
users = list(map(lambda x: str(x), users))
df = pd.DataFrame(sparse_mat, index=users, columns=items)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


         user  item
302     10001  1000
303     10001  1001
304     10001  1002
306     10002  1001
307     10002  1003
309     10003  1001
310     10003  1003
311     10003  1004
313     10004  1005
315     10005  1006
317     10006  1003
318     10006  1004
320     10007  1007
322     10008  1004
324     10009  1008
325     10009  1009
327     10010  1010
328     10010  1000
329     10010  1011
330     10010  1012
331     10010  1013
332     10010  1014
334     10011  1015
335     10011  1016
336     10011  1017
337     10011  1018
338     10011  1019
340     10012  1020
341     10012  1021
343     10013  1022
...       ...   ...
131628  42703  1004
131630  42704  1064
131631  42704  1137
131632  42704  1026
131633  42704  1041
131635  42705  1158
131636  42705  1034
131637  42705  1004
131639  42706  1059
131640  42706  1003
131641  42706  1001
131642  42706  1004
131644  42707  1008
131645  42707  1030
131646  42707  1009
131647  42707  1058
131648  42707  1004
131649  42707  1018


In [10]:
df.to_csv("./../dataset/mat.csv")
df.loc["42612":,:"X1100"].to_csv("./../dataset/test.csv")